In [ ]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 1.1MB 8.2MB/s 
     |████████████████████████████████| 71kB 5.9MB/s 
     |████████████████████████████████| 2.8MB 17.6MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 163kB 39.4MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 5.5MB 30.4MB/s 
     |████████████████████████████████| 1.8MB 36.3MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 133kB 41.1MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.4.0-cp37-none-any.whl size=76208 sha256=ea9368fd828bddaea0d17fa3787195eea3ce8db618ad56aec9c6645e70fd056d
  Stored in directory: /root/.cache/pip/wheels/1a/08/00/1eb79fd13e6095132bf00bd92f803e90797813f40429485461
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=

In [ ]:
#importing libraries and global variables
import pandas as pd
import numpy as np
from glob import glob
import re
from os import listdir
from collections import Counter
import pyterrier as pt

In [ ]:
directories = []
for i in glob("/content/drive/MyDrive/IR/masc_500k_texts/masc_500k_texts/*/"):
  for j in glob(i+'*/'):
    directories.append(j)

data = []
for i in directories:
  for j in listdir(i):
    with open(i+j, "r") as f:
      try:
        d = {'fileName': j, 
             'text': f.read()}
        data.append(d)
      except:
        continue

In [ ]:
len(data)

390

In [ ]:
#we need just the words. so remove all the special characters
regex = r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9 '.]+" #anything that is not a number or alphabet
words = []
for i in data:
  i['text'] = re.sub(regex,' ',i['text'].lower())
  for j in i['text'].split():
    words.append(j.lower())

words = list(set(words))

In [ ]:
len(words)

38311

#pyterrier

In [ ]:
if not pt.started():
  pt.init()
!rm -rf ./iter_index
iter_indexer = pt.IterDictIndexer("./iter_index")

terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [ ]:
def antique_doc_iter():
  for i, d in enumerate(data):
      yield {'docno': i, 'text': d['text'], 'title': d['fileName'],}

In [ ]:
doc_iter = antique_doc_iter()
indexref = iter_indexer.index(doc_iter)

In [ ]:
indexref

<org.terrier.querying.IndexRef at 0x7f79d91883b0 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x55dcd37e2350 at 0x7f79d91d76d0>>

In [ ]:
pt.BatchRetrieve(indexref).transform("presidential election")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  """Entry point for launching an IPython kernel.


,qid,docid,docno,rank,score,query
0,1,1,1,0,6.154093,presidential election
1,1,0,0,1,5.584976,presidential election
2,1,144,144,2,4.894075,presidential election
3,1,213,213,3,4.674847,presidential election
4,1,166,166,4,4.324482,presidential election
5,1,240,240,5,4.128922,presidential election
6,1,95,95,6,3.532532,presidential election
7,1,295,295,7,3.493741,presidential election
8,1,67,67,8,3.271193,presidential election
9,1,136,136,9,3.244861,presidential election


In [ ]:
def search(query):
  results = []
  docno = [int(k) for k in pt.BatchRetrieve(indexref).transform(query).docno]
  doc_iter = antique_doc_iter()
  for i in range(len(docno)):
    results.append(next({'docno':item['docno'],'file':item['title']} for item in doc_iter if item["docno"] in docno))
  return results

In [ ]:
results = search('presidential election')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
results

[{'docno': 0, 'file': '2nd_Gore-Bush.txt'},
 {'docno': 1, 'file': '3rd_Bush-Kerry.txt'},
 {'docno': 47, 'file': 'lists-003-2114716.txt'},
 {'docno': 67, 'file': '9085.txt'},
 {'docno': 95, 'file': 'chapter-10.txt'},
 {'docno': 109, 'file': '1400.txt'},
 {'docno': 120, 'file': 'Effing-Idiot.txt'},
 {'docno': 130, 'file': 'Fermentation_Eminent-Domain.txt'},
 {'docno': 136, 'file': 'A_defense_of_Michael_Moore.txt'},
 {'docno': 144, 'file': 'wsj_0120.txt'},
 {'docno': 166, 'file': '20020731-nyt.txt'},
 {'docno': 196, 'file': 'pirates.txt'},
 {'docno': 213, 'file': 'appalachian1.txt'},
 {'docno': 240, 'file': 'defenders5.txt'},
 {'docno': 256, 'file': 'tweets1.txt'},
 {'docno': 257, 'file': 'tweets2.txt'},
 {'docno': 295, 'file': '111348.txt'}]